In [7]:

import pygame
import math
import random

pygame.init()

BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
PURPLE = (255, 0, 255)

PADDLE_SPEED = 10
BALL_SPEED = 8

WIDTH, HEIGHT = 1000, 800
my_font = pygame.font.SysFont('dejavuserif', 30)

screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Pong")

paddle_img = pygame.image.load('paddle.jpg')
paddle_img = pygame.transform.rotate(paddle_img, 90)

bg = pygame.image.load("bg.jpg")
bg = pygame.transform.scale(bg, (WIDTH, HEIGHT))

class Paddle(pygame.sprite.Sprite):
    def __init__(self, centerx, centery):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.transform.scale(paddle_img, (200, 50))
        self.rect = self.image.get_rect()
        self.rect.centerx = centerx
        self.rect.centery = centery
        self.speed = 0
        self.click = False
        self.score = 0

    def update(self):
        self.rect.centerx += self.speed

class Ball:
    def __init__(self, posx, posy, radius, speed, color):
        self.posx = posx
        self.posy = posy
        self.radius = radius
        self.speed = speed
        self.color = color
        self.x_direction = 1
        self.y_direction = -1
        self.ball = pygame.draw.circle(screen, self.color, (self.posx, self.posy), self.radius)
        # This is true at the beginning of the game and when the ball is
        # freshly respawned after going out of bounds. 
        self.respawned = True
 
    def display(self):
        self.ball = pygame.draw.circle(screen, self.color, (self.posx, self.posy), self.radius)
 
    def update(self):   
        if self.respawned:
            self.posx += self.speed
            return
            
        self.posx += self.speed*self.x_direction
        self.posy += self.speed*self.y_direction

        # if we hit the top or bottom of the screen
        if self.posx <= 0 or self.posx >= WIDTH:
            self.x_direction *= -1

        if self.posy <= 0:
            self.y_direction *= -1
 
    # Used to reflect the ball along the X-axis
    def hit(self):
        self.y_direction *= -1
 
    def getRect(self):
        return self.ball

    def respawn(self, paddle):
        self.posx = paddle.rect.centerx
        self.posy =  HEIGHT - 135
        self.speed = 0
        self.respawned = True
     

class Brick(pygame.sprite.Sprite):
    def __init__(self, centerx, centery, color):
        pygame.sprite.Sprite.__init__(self)
        self.rect = pygame.Rect(0, 0, 100, 20) 
        self.rect.centerx = centerx
        self.rect.centery = centery
        self.image = pygame.Surface([100, 20])
        self.image.fill(color)
        self.alive = True 

    def kill(self):
        self.alive = False        
        self.rect = pygame.Rect(-100, -100, 0, 0) 

def main():
    running = True

    all_sprites = pygame.sprite.Group()
    
    paddle = Paddle(WIDTH // 2, HEIGHT - 100)
    all_sprites.add(paddle)

    brick1 = Brick(WIDTH//2, HEIGHT //2, RED)
    all_sprites.add(brick1)
    brick2 = Brick(WIDTH//2 - 110, HEIGHT //2, PURPLE)
    all_sprites.add(brick2)
    brick3 = Brick(WIDTH//2 - 220, HEIGHT //2, GREEN)
    all_sprites.add(brick3)
    bricks = [brick1, brick2, brick3]
    
    ball = Ball(WIDTH // 2, HEIGHT - 135, 10, 0, WHITE)
    
    while running:
        screen.fill(BLACK)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    paddle.speed = -PADDLE_SPEED
                    if ball.respawned:
                        ball.speed = -PADDLE_SPEED
                if event.key == pygame.K_RIGHT:
                    paddle.speed = PADDLE_SPEED
                    if ball.respawned:
                        ball.speed = +PADDLE_SPEED
            if event.type == pygame.KEYUP:
                if event.key == pygame.K_LEFT:
                    paddle.speed = 0
                    if ball.respawned:
                        ball.speed = 0
                if event.key == pygame.K_RIGHT:
                    paddle.speed = 0
                    if ball.respawned:
                        ball.speed = 0

            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_SPACE:
                    if ball.respawned == False:
                        continue
                    ball.respawned = False
                    ball.speed = BALL_SPEED
                    ball.x_direction = random.uniform(0.5,0.9)
                    ball.y_direction = random.uniform(0.5,0.9)
                
        if pygame.Rect.colliderect(ball.getRect(), paddle.rect):
            ball.hit()
            paddle.score += 1

        for b in bricks:
            if pygame.Rect.colliderect(ball.getRect(), b.rect):
                ball.hit()
                b.kill()
                
        paddle_score_text_surface = my_font.render("Score: " + str(paddle.score), False, (255, 0, 0))

        if ball.posy > HEIGHT:
            ball.respawn(paddle)
        
        ball.update()
        paddle.update()

        screen.blit(bg, (0, 0))
        screen.blit(paddle_score_text_surface, (100, 100))
        
        ball.display()
        all_sprites.draw(screen)
        pygame.display.update()


if __name__ == "__main__":
    main()
    pygame.quit()
 